# Alpha-AGI Business v1 • Colab Notebook

Run a minimal **Alpha-Factory** orchestrator. Works fully offline or upgrades automatically when `OPENAI_API_KEY` is set.

## 1 · Setup

In [ ]:
%%bash
if [ ! -d AGI-Alpha-Agent-v0 ]; then
  git clone --depth 1 https://github.com/MontrealAI/AGI-Alpha-Agent-v0.git
fi
cd AGI-Alpha-Agent-v0
pip -q install fastapi uvicorn requests openai_agents

## 2 · (Optional) Configure your OpenAI API key

In [ ]:
import os, getpass
os.environ['OPENAI_API_KEY'] = getpass.getpass('Enter OpenAI API key (leave blank for offline): ')

## 3 · Launch orchestrator

In [ ]:
import subprocess, sys, time, requests
from IPython.display import display, IFrame
root = 'AGI-Alpha-Agent-v0/alpha_factory_v1'
proc = subprocess.Popen([sys.executable, '-m', 'alpha_factory_v1.backend.orchestrator', '--dev'], cwd=root, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
for _ in range(40):
    try:
        if requests.get('http://localhost:8000/healthz').status_code == 200:
            break
    except Exception:
        time.sleep(0.5)
display(IFrame(src='http://localhost:8000/docs', width='100%', height=600))

## 4 · Post a demo alpha job

In [ ]:
import requests, json
job = {
    'agent': 'planning',
    'pair': 'LME:HG/COMEX:HG',
    'note': 'Deploy copper spread arbitrage strategy'
}
resp = requests.post('http://localhost:8000/agent/planning/trigger', json=job)
print(resp.status_code, resp.text)

## 5 · Graceful shutdown

In [ ]:
proc.terminate(); print('✅ Orchestrator stopped')